In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import read_nml as rnl

from Utils import numerical_utils as nuti


In [ ]:
# Replace with your path
case='correctedZeta'
odir = f"../cases/{case}"

nml_path = f"{odir}/atm_in"

nml = rnl.read_namelist(nml_path)
group_name, active = rnl.choose_active_group(nml)

print(f"Active group: {group_name}")
# Access fields robustly
bnd_topo = active.get('bnd_topo')
ncdata = active.get('ncdata')
scale = active.get('scale_dry_air_mass')
use_topo = active.get('use_topo_file')

print("bnd_topo:", bnd_topo)
print("ncdata:", ncdata)
print("scale_dry_air_mass:", scale)
print("use_topo_file:", use_topo)


In [ ]:
print( active )

In [ ]:

# Open the binary file
with open(f"{odir}/GW.dat", 'rb') as f:
    # Read the dimensions (ncol, nlev) - Fortran writes integers in 4 bytes by default
    ncol, nlev = np.fromfile(f, dtype=np.int32, count=2)
    print(f"Dimensions read from file: ncol={ncol}, nlev={nlev}")
    lat = np.fromfile(f, dtype=np.float64, count=ncol  )                                               #.reshape((ncol, nlev+1), order='F')
    lon = np.fromfile(f, dtype=np.float64, count=ncol  )                                               #.reshape((ncol, nlev+1), order='F')
    sgh = np.fromfile(f, dtype=np.float64, count=ncol  )                                               #.reshape((ncol, nlev+1), order='F')



    
    # Read array (two dimensions), Fortran order
    zm = np.fromfile(f, dtype=np.float64, count=ncol * nlev).reshape((ncol, nlev), order='F')
    # Read  array (two dimensions), Fortran order
    zi = np.fromfile(f, dtype=np.float64, count=ncol * (nlev+1) ).reshape((ncol, nlev+1), order='F')

    uu = np.fromfile(f, dtype=np.float64, count=ncol * nlev).reshape((ncol, nlev), order='F')
    vv = np.fromfile(f, dtype=np.float64, count=ncol * nlev).reshape((ncol, nlev), order='F')

    # read booleans (4 bytes each)
    flags_int = np.fromfile(f, dtype=np.int32, count=2)
    use_gw_rdg_beta , use_gw_movmtn_pbl = flags_int != 0  # convert to booleans

    print( f"Boolean use_gw_rdg_beta and use_gw_movmtn_pbl vars => {use_gw_rdg_beta} {use_gw_movmtn_pbl}" )
    
    tau_a = np.fromfile(f, dtype=np.float64, count=ncol * (nlev+1) ).reshape((ncol, nlev+1), order='F')
    tau_b = np.fromfile(f, dtype=np.float64, count=ncol * (nlev+1) ).reshape((ncol, nlev+1), order='F')

    bwv = np.fromfile(f, dtype=np.float64, count=ncol  )                                               #.reshape((ncol, nlev+1), order='F')
    tlb = np.fromfile(f, dtype=np.float64, count=ncol  )                                               #.reshape((ncol, nlev+1), order='F')
    wbr = np.fromfile(f, dtype=np.float64, count=ncol  )                                               #.reshape((ncol, nlev+1), order='F')

    ubmsrc = np.fromfile(f, dtype=np.float64, count=ncol  )                                               #.reshape((ncol, nlev+1), order='F')
    nsrc = np.fromfile(f, dtype=np.float64, count=ncol  )                                               #.reshape((ncol, nlev+1), order='F')

    tauoro = np.fromfile(f, dtype=np.float64, count=ncol  )                                               #.reshape((ncol, nlev+1), order='F')
    taudsw = np.fromfile(f, dtype=np.float64, count=ncol  )                                               #.reshape((ncol, nlev+1), order='F')
    
    ubm = np.fromfile(f, dtype=np.float64, count=ncol * (nlev) ).reshape((ncol, nlev), order='F')
    tau = np.fromfile(f, dtype=np.float64, count=ncol * (nlev+1) ).reshape((ncol, nlev+1), order='F')
    tau_diag = np.fromfile(f, dtype=np.float64, count=ncol * (nlev+1) ).reshape((ncol, nlev+1), order='F')

    if (use_gw_movmtn_pbl==True):
        ubm_movmtn  = np.fromfile(f, dtype=np.float64, count=ncol * (nlev) ).reshape((ncol, nlev), order='F')
        vort_movmtn = np.fromfile(f, dtype=np.float64, count=ncol * (nlev) ).reshape((ncol, nlev), order='F')
        tau_movmtn  = np.fromfile(f, dtype=np.float64, count=ncol * (nlev+1) ).reshape((ncol, nlev+1), order='F')

# 


In [ ]:
print( tau.shape )

In [ ]:

print( ncdata )
print(active['ncdata_type'])


In [ ]:
#
X=xr.open_dataset( ncdata )

In [ ]:
if (active['ncdata_type'] == 'XY_DATA' ):
    dims=np.array( X.dims.values )
    nt = X.sizes['time'] 
    nz = X.sizes['lev'] 
    ny = X.sizes['lat'] 
    nx = X.sizes['lon'] 
    print( X.upwp.values.shape )
    lat_xy = lat.reshape( [nx,ny] )
    sgh_xy = sgh.reshape( [ny,nx] )
    # This reshaping seems to work 
    zm_xy = zm.reshape( [ny,nx,nz] )
    uu_xy = uu.reshape( [ny,nx,nz] )
    vv_xy = vv.reshape( [ny,nx,nz] )
    zeta_xy = vort_movmtn.reshape( [ny,nx,nz] )
    

In [ ]:
if (active['ncdata_type'] == 'XY_DATA' ):
    lon1 = X.lon.values
    lat1 = X.lat.values
    upwp=X.upwp.values
    vpwp=X.vpwp.values
    uu_m=X.U.values
    vv_m=X.V.values
    
    mpwp=np.sqrt( upwp**2 + vpwp**2 )

    zeta_m = np.zeros(  (nt,nz,ny,nx) )
    for z in np.arange( nz ):
        zeta_m[0,z,:,:] = nuti. Sphere_Curl2( f_x=uu_m[0,z,:,:], f_y=vv_m[0,z,:,:], lat=lat1, lon=lon1, wrap=True )
        #print( z )


In [ ]:
fig,axs=plt.subplots( 1,2, figsize=(33,8) )

t,z=0,80
levels=1e-3 * np.linspace( -1,1,num=21)
cmap='bwr'

ax=axs[0]
c=ax.contourf( lon1, lat1, zeta_m[t,z,:,:] , levels= levels, cmap=cmap ) # vort_movmtn[:,90] )
ax.set_title( ' voriticy from Python' )
plt.colorbar(c)
ax=axs[1]
c=ax.contourf( lon1, lat1, zeta_xy[:,:,z] , levels= levels, cmap=cmap  ) # vort_movmtn[:,90] )
ax.set_title( 'vorticity from Fortran' )
plt.colorbar(c)


In [ ]:
#plt.scatter( zeta_m[t,z,:,:].flatten() , zeta_xy[:,:,z].flatten() )

plt.plot( zeta_m[t,z,:,:].flatten() )
plt.plot( zeta_xy[:,:,z].flatten() )
plt.plot( zeta_m[t,z,:,:].flatten() - zeta_xy[:,:,z].flatten() )

